## Assorted Azure functions for Blob Storage
This is the most up to date "basic documentation" for 2020. The official Microsoft docs are for the old implementation of `blob-storage`.

Pre-req:
```
pip3 install azure-storage
pip3 install pyodbc
```

https://docs.microsoft.com/en-us/python/api/azure-storage-blob/azure.storage.blob.baseblobservice.baseblobservice?view=azure-python-previous

In [1]:
# get credentials on WSL environment securely
azKey = !az storage account keys list --resource-group RESOURCE_GROUP_NAME --account-name ACCOUNT_NAME
azStr = !az storage account show-connection-string --name NAME

key = azKey[4].strip(' "value:": ')
cnxnStr = azStr[1].strip(' "connectionString": ')

In [2]:
# azure storage API for image repo
import os
from azure.storage.blob import BlockBlobService, baseblobservice

# constants
container_name = 'test-images'
# production contName = 'images'
account_name = 'gs1images'

URL = f'https://{account_name}.blob.core.windows.net'
DIR = f'{container_name}'

# connect to block blob service
block_service = BlockBlobService(
    account_key=key,
    connection_string=cnxnStr
)

# connect to base blob service
base_service = baseblobservice.BaseBlobService(
    account_key=key,
    connection_string=cnxnStr
)

### Uploading from local path

In [ ]:
# upload sample barcode stuff from an images directory
for img_name in os.listdir(DIR):
    print(f"[Uploading]: {img_name}")
    fpath = os.path.join(DIR, img_name)
    block_service.create_blob_from_path(container_name, img_name, fpath)

### List all blobs in the blob storage

In [ ]:
# get/print a list of all images in the blob
generator = base_service.list_blobs(container_name)
print([i.name for i in generator])

### Deleting blobs from storage

```python
# delete all blobs in an iterative manner
generator = base_service.list_blobs(container_name)
for img in generator:
    base_service.delete_blob(container_name, img.name)
```

### Downloading blobs from storage

In [ ]:
# download all the blobs in a specified container
DOWNLOAD = f'{container_name}-download'
CWD = os.path.join(os.getcwd(), DOWNLOAD)
generator = block_service.list_blobs(container_name)

# create container_name-download
os.makedirs(CWD, exist_ok=True)

for img in generator:
    print(f"Downloading {img.name}")
    
    # check if the path includes a structured folder
    if '/' in img.name or '\\' in img.name:
        head, tail = os.path.split(img.name)
        print(f"There is a folder path. Head: {head}, Tail: {tail}.")
        path = os.path.isdir(os.path.join(CWD, head))
        
        # if it doesnt exist, create it
        if not path:
            print("Creating the folder path as it does not exist.")
            os.makedirs(os.path.join(CWD, head), exist_ok=True)
        
        # download file
        block_service.get_blob_to_path(container_name, img.name, os.path.join(path, tail))
    else:
        block_service.get_blob_to_path(container_name, img.name, os.path.join(CWD, img.name))